# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f73208cc190>
├── 'a' --> tensor([[ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088]])
└── 'x' --> <FastTreeValue 0x7f73208ccee0>
    └── 'c' --> tensor([[ 3.4909, -2.0214, -0.8582,  0.2418],
                        [-0.3799, -0.4379, -0.4376,  1.0057],
                        [-1.0911,  0.1900,  0.8893,  2.0028]])

In [4]:
t.a

tensor([[ 0.4960, -0.0107, -0.1843],
        [ 0.4342, -0.1665,  0.7088]])

In [5]:
%timeit t.a

73.9 ns ± 0.866 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f73208cc190>
├── 'a' --> tensor([[-1.3389,  0.7729,  1.2049],
│                   [ 0.5881,  0.9834, -0.8464]])
└── 'x' --> <FastTreeValue 0x7f73208ccee0>
    └── 'c' --> tensor([[ 3.4909, -2.0214, -0.8582,  0.2418],
                        [-0.3799, -0.4379, -0.4376,  1.0057],
                        [-1.0911,  0.1900,  0.8893,  2.0028]])

In [7]:
%timeit t.a = new_value

77.8 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088]]),
    x: Batch(
           c: tensor([[ 3.4909, -2.0214, -0.8582,  0.2418],
                      [-0.3799, -0.4379, -0.4376,  1.0057],
                      [-1.0911,  0.1900,  0.8893,  2.0028]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4960, -0.0107, -0.1843],
        [ 0.4342, -0.1665,  0.7088]])

In [11]:
%timeit b.a

74.7 ns ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3258, -0.2371,  0.0467],
               [-1.9247,  0.3447, -0.5207]]),
    x: Batch(
           c: tensor([[ 3.4909, -2.0214, -0.8582,  0.2418],
                      [-0.3799, -0.4379, -0.4376,  1.0057],
                      [-1.0911,  0.1900,  0.8893,  2.0028]]),
       ),
)

In [13]:
%timeit b.a = new_value

637 ns ± 4.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.02 µs ± 9.24 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.2 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

292 µs ± 8.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

275 µs ± 3.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7320860550>
├── 'a' --> tensor([[[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]],
│           
│                   [[ 0.4960, -0.0107, -0.1843],
│                    [ 0.4342, -0.1665,  0.7088]]])
└── 'x' --> <FastTreeValue 0x7f73208161f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

51.7 µs ± 352 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f73248e5be0>
├── 'a' --> tensor([[ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088],
│                   [ 0.4960, -0.0107, -0.1843],
│                   [ 0.4342, -0.1665,  0.7088]])
└── 'x' --> <FastTreeValue 0x7f727c1ca5e0>
    └── 'c' --> tensor([[ 3.4909, -2.0214, -0.8582,  0.2418],
                        [-0.3799, -0.4379, -0.4376,  1.0057],
                 

In [23]:
%timeit t_cat(trees)

47.1 µs ± 919 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.5 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]],
       
               [[ 0.4960, -0.0107, -0.1843],
                [ 0.4342, -0.1665,  0.7088]]]),
    x: Batch(
           c: tensor([[[ 3.4909, -2.0214, -0.8582,  0.2418],
                       [-0.3799, -0.4379, -0.4376,  1.0057],
                       [-1.0911,  0.1900,  0.8893,  2.0028]],
         

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088],
               [ 0.4960, -0.0107, -0.1843],
               [ 0.4342, -0.1665,  0.7088]]),
    x: Batch(
           c: tensor([[ 3.4909, -2.0214, -0.8582,  0.2418],
                      [-0.3799, -0.4379, -0.4376,  1.0057],
                      [-1.0911,  0.1900,  0.8893,  2.0028],
                      [ 3.4909, -2.0214, -0.8582,  0.2418],
                      [-0.3799, -

In [28]:
%timeit Batch.cat(batches)

188 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

614 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
